## Final Project Submission

Please fill out:
* Student name: Cameryn
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Victor Geislinger
* Blog post URL: 


# Importing important packages
For this project, the pandas, numpy, seaborn, matplotlib, and sqlite3 packages are needed and imported. The matplotlib is also declared as inline to avoid any potential issues in the future.

In [ ]:
import pandas as pd # for dataframes
import numpy as np
import seaborn as sns # for making our charts more readable
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

%matplotlib inline

# Importing all data as CSV and TSV files

First, it is important to bring all of the data into a usable, readable format, in this case, pandas dataframes.

**Note that one file, 'rt.reviews.tsv' is encoded differently and thus needed to be converted using the correct encoding declaration.**

In [ ]:
#Import data
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv')
im_n_basics_df = pd.read_csv('zippedData/imdb.name.basics.csv')
im_akas_df = pd.read_csv('zippedData/imdb.title.akas.csv')
im_basics_df = pd.read_csv('zippedData/imdb.title.basics.csv')
im_crew_df = pd.read_csv('zippedData/imdb.title.crew.csv')
im_principals_df = pd.read_csv('zippedData/imdb.title.principals.csv')
im_ratings_df = pd.read_csv('zippedData/imdb.title.ratings.csv')
rt_movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv', sep='\t')
rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv', sep='\t', encoding = 'latin1')
tmdb_movies_df = pd.read_csv('zippedData/tmdb.movies.csv')
tn_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv')

After importing them, the .info() of each dataframe was checked in order to determine the data types of each column and determine what needed to be done in the below cells. That is not shown below for each dataframe, simply due to the amount of space that does take, but it is shown for the first dataframe, bom_df.

**Note the 'foreign_gross' column's type. This will prevent future statistical analysis, which is addressed in the future.**

In [ ]:
bom_df.info()

# Data Cleaning
In order to appropriately understand and interpret each dataframe in the context of the others, it is important to clean andappropriately format each column, removing unhelpful and ill-formated data and replacing where possible.

### Changing Column Type
The issue noted previously with bom_bf['foreign_gross'], is common across a few of our dataframes, thus the below function has been written to prevent unnecessary repetition.

In [ ]:

def column_type_changer(column, df):
    """Returns stripped, numeric column from object-type column
       Args:
           column: Column to be converted.
           df: DataFrame that column is located in.
       Returns:
           Series data converted to numeric data type"""
    
    df[column] = df[column].str.replace('$','') # Removing non-numeric chars
    df[column] = df[column].str.replace(',','') # Removing non-numeric chars
    df[column] = df[column].str.replace(' minutes','') # Removing non-numeric chars
    df[column] = df[column].apply(pd.to_numeric) # Converts to numbers
    return df[column]

In [ ]:
bom_df['foreign_gross'] = column_type_changer('foreign_gross', bom_df)

In [ ]:
print(bom_df.info())
print(bom_df.describe())
bom_df.head()

Note the number of null objects in the 'foreign_gross' column. This could be attributed to a number of things, though often it could simply be that the film wasn't released abroad. Without context, it may cause more problems that it would solve to replace that information. Further exploration of the other data may be necessary.

In [ ]:
im_n_basics_df.info()
im_n_basics_df.head()

In [ ]:
im_akas_df.info()
im_akas_df.head()

### Merging DataFrames
In order to better utilize this data, the ratings dataframe will be merged with the basics dataframe, as the ratings dataframe does not contain any keys that otherwise would refer to other dataframes in future data manipulation. The crew dataframe will also be merged into the dataframe, utilizing the tconstant column in order to maintain consistency, and to allow future joins and subqueries to be much more simple.

**While this does remove some lines from the data, as the client is clearly attempting to determine what the best option is to enter the film business, requiring ratings seems to be a good choice.**

In [ ]:
im_crew_df.head() # Quick preview of crew dataframe

In [ ]:
im_ratings_df.head() # Quick preview of ratings dataframe

In [ ]:
im_basics_df = pd.merge(im_basics_df, im_ratings_df, on='tconst')
im_basics_df = pd.merge(im_basics_df, im_crew_df, on='tconst')
im_basics_df.head()
im_basics_df.info()

In [ ]:
im_principals_df.info()
im_principals_df.head()

In [ ]:
rt_movie_info_df.info()
rt_movie_info_df['box_office'] = column_type_changer('box_office',rt_movie_info_df)
rt_movie_info_df['runtime'] = column_type_changer('runtime',rt_movie_info_df)
rt_movie_info_df.head()

In [ ]:
rt_reviews_df.info() #use this data to potentially invite specific reviewers
rt_reviews_df.head()

In [ ]:
tmdb_movies_df.head()

In [ ]:
tn_budgets_df['production_budget'] = column_type_changer('production_budget', tn_budgets_df)
tn_budgets_df['domestic_gross'] = column_type_changer('domestic_gross', tn_budgets_df)
tn_budgets_df['worldwide_gross'] = column_type_changer('worldwide_gross', tn_budgets_df)
tn_budgets_df.head()

# Creating a Database
It is necessary to create a database first to tie all of the previous data together and to allow for queries.

In [ ]:
# Create the movies database
conn = sqlite3.connect('movies.sqlite')
cur = conn.cursor()

In the below cells, the dataframes will be converted to sqlite tables in order to be better queried. This will allow for joining on various columns/keys, as will be explained in the future.

In [ ]:
engine = create_engine('sqlite://', echo=False)
im_basics_df.to_sql('movies', con=engine)

engine.execute("""SELECT * 
                from movies;""").fetchall()

In [ ]:
im_n_basics_df.to_sql('workers',con=engine)
engine.execute("""SELECT * 
                from workers;""").fetchall()

In [ ]:
im_akas_df.to_sql('movieNames',con=engine)
engine.execute("""SELECT * 
                from movieNames;""").fetchall()